In this notebook, we gather two pieces of annotation: one csv is the 2K groundtruth, one is our snorkel gen output. See how the PP precision goes.
    
    

In [4]:
import pathlib
import pandas as pd

groundtruth_path = pathlib.Path.cwd().joinpath('data', 'annotations_word-level_all-to-date-2018-4-25.xlsx')
df_groundtruth = pd.read_excel(groundtruth_path)

# first check if globalPsn is the same 

for paperID, paperData in df_groundtruth.groupby('paperID'):
    paperData = paperData.sort_values(by="globalPsn")
    
#     # get the first row first
#     # we're going to switch if the current highlight is different from the last one
#     firstRow = paperData.iloc[0]
#     lastType = firstRow['winningHighlight']
#     segmentWords = [str(firstRow['content'])]
#     sequence = 1
#     #secondRowOnwards = paperData.iloc[1:]
    
#     for index, row in paperData.iloc[1:].iterrows():
        
#         # get currentType
#         currentType = row['winningHighlight']
#         if paperID=="wos_cs_507":
#             print(currentType,lastType)




156

159

216

184

170

140

129

154

141

229

143

156

131

186

173

164

137

130

179

130

177

155

92

167

176

174

216

161

137

159

172

164

114

157

135

166

223

137

159

114

99

153

162

140

167

143

140

160

209

167

84

146

66

218

198

273

148

121

112

95

181

92

105

208

92

109

130

218

114

173

168

350

251

171

175

288

186

231

169

167

156

91

114

124

156

150

195

216

150

113

176

188

244

42

184

201

230

169

55

132

179

227

134

82

274

75

199

126

102

138

107

210

97

165

146

58

189

228

179

175

119

208

155

184

198

137

265

167

126

171

185

171

178

211

299

176

197

165

229

288

109

173

156

306

139

478

192

127

172

170

106

187

161

150

210

161

100

270

164

197

388

117

254

151

263

170

179

220

115

181

299

120

177

46

289

220

328

200

82

289

19

219

255

261

147

273

221

188

88

203

231

230

129

260

266

184

134

64

190

137

150

277

246

172


191

176

133

204

277

143

273

159

261

264

247

238

94

199

175

219

92

120

101

122

119

208

222

161

217

204

184

162

259

189

120

153

190

130

213

226

197

191

195

237

230

155

261

142

199

164

211

155

230

131

295

270

256

239

175

188

131

170

186

177

140

151

301

189

275

221

213

139

178

531

275

175

188

143

172

154

177

190

122

103

163

174

146

159

175

184

180

173

248

167

132

207

188

165

174

25

241

187

306

143

238

178

218

140

188

123

194

187

267

179

267

127

171

221

216

139

151

202

211

180

39

149

185

213

186

210

163

222

117

246

164

164

181

150

136

142

226

118

193

142

103

177

192

151

151

159

287

105

76

239

160

117

154

181

165

333

176

128

211

241

106

186

235

173

131

201

259

127

234

152

145

168

224

189

252

166

120

223

167

262

195

149

161

100

134

218

131

159

143

178

201

168

182

216

206

163

72

120

281

332

210

1

In [7]:
sub_dfs=[]
for train_bucket in range(9):
    sub_dfs+=[pd.read_csv(open("data/annotations_2k/split"+str(train_bucket)+".csv","r"))]
df = pd.concat(sub_dfs)
df=df.drop(columns=['Unnamed: 0'])
df.to_csv(open("data/annotations_2k/joint_split.csv","w"))

df2=df.copy()
import ast
import numpy as np

def transform_gm_label(x):
    x=ast.literal_eval(x)
    if abs(x[0]-x[1])<1e-4 and abs(x[1]-x[2])<1e-4:
        return [0,0,1]
    elif np.argmax(x)==0:
        return [1,0,0]
    return [0,1,0]

df2['winningHighlight'] = df2['winningHighlight'].apply(lambda x: transform_gm_label(x))
df2.to_csv(open("data/annotations_2k/joint_split_hard_as_soft.csv","w"))

        Unnamed: 0 winningHighlight     paperID  globalPsn      content
0                0        [0, 0, 1]    2K_dev_0          1       Online
1                1        [0, 0, 1]    2K_dev_0          2  communities
2                2        [0, 0, 1]    2K_dev_0          3         have
3                3        [0, 0, 1]    2K_dev_0          4          the
4                4        [0, 0, 1]    2K_dev_0          5    potential
5                5        [0, 0, 1]    2K_dev_0          6           to
6                6        [1, 0, 0]    2K_dev_0          7           be
7                7        [1, 0, 0]    2K_dev_0          8   supportive
8                8        [1, 0, 0]    2K_dev_0          9            ,
9                9        [1, 0, 0]    2K_dev_0         10        cruel
10              10        [1, 0, 0]    2K_dev_0         11            ,
11              11        [1, 0, 0]    2K_dev_0         12           or
12              12        [1, 0, 0]    2K_dev_0         13     a

In [16]:
label_level_groundtruth_path = pathlib.Path.cwd().joinpath('data', 'annotations_label-level_all-to-date-2018-4-25-WithTitle.csv')
df_label_level = pd.read_csv(label_level_groundtruth_path)

actual_paperID=df_label_level["paperID"].values

In [40]:
import numpy as np
import re
from collections import defaultdict
from sklearn.metrics import precision_recall_fscore_support

validTypes = set(['Method', 'Background', 'Findings', 'Mechanism', 'Purpose'])
validTypesMapping=defaultdict(lambda:2)
validTypesMapping["Method"]=2
validTypesMapping["Background"]=2
validTypesMapping["Findings"]=2
validTypesMapping["Mechanism"]=1
validTypesMapping["Purpose"]=0

In [84]:
# gen_path = pathlib.Path.cwd().joinpath('data', 'annotations_2k','joint_split_hard_as_soft.csv')
# gen_path = pathlib.Path.cwd().joinpath('data', 'annotations_2k','joint_split.csv')
for threshold in [0.33, 0.4, 0.5, 0.6, 0.7, 0.75, 0.8, 0.85, 0.9, 0.95]:
# for threshold in [0.6]:
#     print("working on thrshold of ",threshold)
    gen_path = pathlib.Path.cwd().joinpath('data', 'annotations_2k','joint_split_hard_as_soft_break_tie_'+str(threshold)+".csv")
    df_gen = pd.read_csv(gen_path)
    y_true=[]
    y_pred=[]
    for paperID, paperData in df_gen.groupby('paperID'):
        if int(paperID.split("2K_dev_")[1])>=50:
#             print(paperID,"disqualified")
            continue
        else:
            print(paperID,"qualified")
        paperData = paperData.sort_values(by="globalPsn")

        groundtruth_gb=df_groundtruth.groupby('paperID')
        paperData_groundtruth=groundtruth_gb.get_group(actual_paperID[int(paperID.split("2K_dev_")[1])]).sort_values(by="globalPsn")


        contents=paperData_groundtruth['content'].tolist()
        aspects=paperData_groundtruth['winningHighlight'].tolist()
        sequences=[]

        if len(contents)!=len(aspects):
            print("Error, contents' length not equal aspect tags' length")
            continue

        # this handles cases like "however," where the comma is appended
        for pair in zip(contents,aspects):
            content=re.sub(r"\s+","",str(pair[0]))
            if len(content)==0:
                continue
            aspect=validTypesMapping[pair[1]]
            
            if re.match("\d",content):
                sequences+=[(content,aspect)]
            elif content=="e.g.,":
                sequences+=[("e.g.",aspect)]
                sequences+=[(",",aspect)]
            else:
                output=list(re.findall(r'(\w+)(\W+)*', content))
                token_list=(list(sum(output,())))
                for token in token_list:
                    if len(token.strip())>0:
                        sequences+=[(token,aspect)]
                if (len(token_list)==0) and content!="'":
                    sequences+=[(content,aspect)]
            
            
#             if ":"==content[-1] and "-" in content:
#                 sequences+=[(content.split(":")[0].split("-")[0],aspect)]
#                 sequences+=[("-",aspect)]
#                 sequences+=[(content.split(":")[0].split("-")[1],aspect)]
#                 sequences+=[(":",aspect)] 
#             elif ","==content[-1] and "-" in content:
#                 sequences+=[(content.split(",")[0].split("-")[0],aspect)]
#                 sequences+=[("-",aspect)]
#                 sequences+=[(content.split(",")[0].split("-")[1],aspect)]
#                 sequences+=[(",",aspect)]
#             elif len(content)>1 and ","==content[-1]:
#                 sequences+=[(content[:-1],aspect)]
#                 sequences+=[(",",aspect)]
#             elif len(content)>1 and ":"==content[-1]:
#                 sequences+=[(content[:-1],aspect)]
#                 sequences+=[(":",aspect)]
#             elif len(content)>1 and "-" in content:
#                 sequences+=[(content.split("-")[0],aspect)]
#                 sequences+=[("-",aspect)]
#                 sequences+=[(content.split("-")[1],aspect)]
#             else:
#                 sequences+=[(content,aspect)]

        contents_gen=paperData['content'].tolist()
        aspects_gen=paperData['winningHighlight'].tolist()
        sequences_gen=[]
        for pair in zip(contents_gen,aspects_gen):
            content=re.sub(r"\s+","",str(pair[0]))
            if len(content)==0:
                continue
            gen_label=ast.literal_eval(pair[1])
            aspect=np.argmax(gen_label)
            if abs(gen_label[0]-gen_label[1])<1e-3 and abs(gen_label[1]-gen_label[2])<1e-3:
                aspect=4
            if content=="'":
                continue
            sequences_gen+=[(content,aspect)]
        if len(sequences)!=len(sequences_gen):
#             print(str(sequences))
#             input(str(sequences_gen))
#             print("unmatched once")
            continue
    #     print(sequences_gen)
    #     break
        y_true+=[pair[1] for pair in sequences]
        y_pred+=[pair[1] for pair in sequences_gen]
        print([(contents_gen[i],sequences[i][1],sequences_gen[i][1]) for i in range(len(sequences_gen))])
#         print()

    y_true=np.asarray(y_true)
    y_pred=np.asarray(y_pred)
#     print((y_pred==4).sum())
    metrics=precision_recall_fscore_support(y_true, y_pred, average=None,labels=[0,1,2])
    line=str(threshold)+" "
    for i in range(3):
        line+=str(metrics[0][i])+" "+str(metrics[1][i])+" "+str(metrics[2][i])+" "
    print(line)
#     print()



2K_dev_0 qualified
[('Online', 2, 4), ('communities', 2, 4), ('have', 2, 4), ('the', 2, 4), ('potential', 2, 4), ('to', 2, 4), ('be', 2, 0), ('supportive', 2, 0), (',', 2, 0), ('cruel', 2, 0), (',', 2, 0), ('or', 2, 0), ('anywhere', 2, 0), ('in', 2, 0), ('between', 2, 0), ('.', 2, 0), ('The', 2, 4), ('development', 2, 4), ('of', 2, 4), ('positive', 2, 4), ('norms', 2, 4), ('for', 2, 4), ('interaction', 2, 4), ('can', 2, 4), ('help', 2, 4), ('users', 2, 4), ('build', 2, 4), ('bonds', 2, 4), (',', 2, 4), ('grow', 2, 4), (',', 2, 4), ('and', 2, 4), ('learn', 2, 4), ('.', 2, 4), ('Using', 2, 4), ('millions', 2, 4), ('of', 2, 4), ('messages', 2, 4), ('sent', 2, 4), ('in', 2, 4), ('Twitch', 2, 4), ('chatrooms', 2, 4), (',', 2, 4), ('we', 0, 0), ('explore', 0, 0), ('the', 0, 0), ('effectiveness', 0, 0), ('of', 0, 0), ('methods', 0, 0), ('for', 0, 0), ('encouraging', 0, 0), ('and', 0, 0), ('discouraging', 0, 0), ('specific', 0, 0), ('behaviors', 0, 0), (',', 0, 0), ('including', 0, 0), ('takin

[('Feedback', 2, 0), ('is', 2, 0), ('an', 2, 0), ('important', 2, 0), ('component', 2, 0), ('of', 2, 0), ('the', 2, 0), ('design', 2, 0), ('process', 2, 0), (',', 2, 0), ('but', 2, 0), ('gaining', 0, 0), ('access', 0, 0), ('to', 0, 0), ('high', 0, 0), ('-', 0, 0), ('quality', 0, 0), ('critique', 0, 0), ('outside', 0, 0), ('a', 0, 0), ('classroom', 0, 0), ('or', 0, 0), ('firm', 0, 0), ('is', 0, 0), ('challenging', 0, 0), ('.', 0, 0), ('We', 1, 1), ('present', 1, 1), ('CrowdCrit', 1, 1), (',', 1, 1), ('a', 1, 1), ('web', 1, 1), ('-', 1, 1), ('based', 1, 1), ('system', 1, 1), ('that', 1, 1), ('allows', 1, 1), ('designers', 1, 1), ('to', 1, 1), ('receive', 1, 0), ('design', 1, 0), ('critiques', 1, 0), ('from', 1, 0), ('non', 1, 0), ('-', 1, 0), ('expert', 1, 0), ('crowd', 1, 0), ('workers', 1, 0), ('.', 2, 0), ('We', 2, 2), ('evaluated', 2, 2), ('CrowdCrit', 2, 2), ('in', 2, 2), ('three', 2, 2), ('studies', 2, 2), ('focusing', 2, 2), ('on', 2, 2), ('the', 2, 2), ('designer', 2, 2), ("'s", 

[('In', 1, 2), ('this', 1, 2), ('paper', 1, 2), (',', 1, 2), ('we', 1, 1), ('present', 1, 1), ('the', 1, 1), ('Group', 1, 1), ('Context', 1, 1), ('Framework', 1, 1), ('(', 1, 1), ('GCF', 1, 1), (')', 1, 1), (',', 1, 1), ('a', 1, 2), ('general', 1, 2), ('-', 1, 1), ('purpose', 1, 1), ('toolkit', 1, 1), ('that', 1, 1), ('allows', 0, 1), ('mobile', 0, 1), ('devices', 0, 1), ('to', 0, 1), ('opportunistically', 0, 1), ('share', 0, 1), ('contextual', 0, 1), ('information', 0, 1), ('.', 2, 1), ('GCF', 1, 1), ('provides', 1, 1), ('a', 1, 1), ('standardized', 1, 1), ('way', 1, 1), ('for', 1, 1), ('developers', 1, 1), ('to', 1, 1), ('request', 1, 0), ('contextual', 1, 0), ('data', 1, 0), ('for', 1, 0), ('their', 1, 0), ('applications', 1, 0), ('.', 2, 0), ('The', 1, 4), ('framework', 1, 4), ('then', 1, 4), ('intelligently', 1, 4), ('groups', 1, 4), ('with', 1, 4), ('other', 1, 4), ('devices', 1, 0), ('to', 1, 0), ('satisfy', 1, 0), ('these', 1, 0), ('requirements', 1, 0), ('.', 2, 0), ('Through'

2K_dev_2 qualified
2K_dev_20 qualified
2K_dev_21 qualified
[('When', 2, 4), ('health', 2, 4), ('services', 2, 4), ('involve', 2, 4), ('long', 2, 4), ('-', 2, 4), ('term', 2, 4), ('treatment', 2, 4), ('over', 2, 4), ('months', 2, 4), ('or', 2, 4), ('years', 2, 1), (',', 2, 1), ('providers', 2, 1), ('have', 2, 1), ('the', 2, 1), ('ability', 2, 1), (',', 2, 1), ('not', 2, 1), ('present', 2, 1), ('in', 2, 1), ('acute', 2, 1), ('emergency', 2, 1), ('care', 2, 1), (',', 2, 1), ('to', 2, 1), ('collaboratively', 2, 1), ('reflect', 2, 1), ('on', 2, 1), ('clients', 2, 1), ("'", 2, 1), ('changing', 2, 1), ('health', 2, 1), ('data', 2, 1), ('and', 2, 1), ('adjust', 2, 1), ('interventions', 2, 1), ('.', 0, 4), ('In', 0, 4), ('this', 0, 4), ('paper', 0, 4), (',', 0, 4), ('we', 0, 4), ('discuss', 0, 4), ('temporality', 0, 4), ('as', 0, 4), ('a', 0, 4), ('factor', 0, 4), ('in', 0, 4), ('the', 0, 4), ('design', 0, 4), ('of', 0, 4), ('health', 0, 4), ('information', 0, 4), ('technology', 0, 4), ('.', 2,

[('Collective', 2, 0), ('intelligence', 2, 0), ('(', 2, 0), ('CI', 2, 0), (')', 2, 0), (',', 2, 0), ('a', 2, 0), ('group', 2, 0), ("'s", 2, 0), ('capacity', 2, 0), ('to', 2, 0), ('perform', 2, 0), ('a', 2, 0), ('wide', 2, 0), ('variety', 2, 0), ('of', 2, 0), ('tasks', 2, 0), (',', 2, 0), ('is', 2, 0), ('a', 2, 0), ('key', 2, 0), ('factor', 2, 0), ('in', 2, 0), ('successful', 2, 0), ('collaboration', 2, 0), ('.', 2, 0), ('Group', 0, 4), ('composition', 0, 4), (',', 0, 4), ('particularly', 0, 4), ('diversity', 0, 4), ('and', 0, 4), ('member', 0, 4), ('social', 0, 4), ('perceptiveness', 0, 4), (',', 0, 4), ('are', 0, 4), ('consistent', 0, 4), ('predictors', 0, 4), ('of', 0, 4), ('CI', 0, 4), (',', 0, 4), ('but', 0, 0), ('we', 0, 0), ('have', 0, 0), ('limited', 0, 0), ('knowledge', 0, 0), ('about', 0, 0), ('the', 0, 0), ('mechanisms', 0, 0), ('underlying', 0, 0), ('their', 0, 0), ('effects', 0, 0), ('.', 0, 0), ('To', 0, 4), ('address', 0, 0), ('this', 0, 0), ('gap', 0, 0), (',', 0, 0), ('

...

[('A', 2, 4), ('challenge', 2, 4), ('for', 2, 4), ('many', 2, 4), ('online', 2, 4), ('production', 2, 4), ('communities', 2, 4), ('is', 2, 4), ('to', 2, 4), ('direct', 2, 4), ('their', 2, 4), ('members', 2, 4), ('to', 2, 4), ('accomplish', 2, 4), ('tasks', 2, 0), ('that', 2, 0), ('are', 2, 0), ('important', 2, 4), ('to', 2, 4), ('the', 2, 4), ('group', 2, 4), (',', 2, 4), ('even', 2, 4), ('when', 2, 0), ('these', 2, 0), ('tasks', 2, 0), ('may', 2, 4), ('not', 2, 4), ('match', 2, 4), ('individual', 2, 4), ('members', 2, 4), ("'", 2, 4), ('interests', 2, 4), ('.', 0, 4), ('Here', 0, 4), ('we', 0, 4), ('investigate', 0, 4), ('how', 0, 4), ('combining', 0, 4), ('group', 0, 4), ('identification', 0, 4), ('and', 0, 4), ('direction', 0, 4), ('setting', 0, 4), ('can', 0, 4), ('motivate', 0, 4), ('volunteers', 0, 4), ('in', 0, 4), ('online', 0, 4), ('communities', 0, 4), ('to', 0, 4), ('accomplish', 0, 4), ('tasks', 0, 4), ('important', 0, 4), ('to', 0, 4), ('the', 0, 4), ('success', 0, 4), ('o

[('HomeProxy', 0, 4), ('is', 0, 4), ('a', 0, 4), ('research', 0, 4), ('prototype', 0, 4), ('that', 0, 4), ('uses', 0, 4), ('a', 0, 4), ('physical', 0, 4), ('proxy', 0, 4), ('to', 0, 4), ('support', 0, 4), ('video', 0, 4), ('messaging', 0, 4), ('at', 0, 4), ('home', 0, 4), ('among', 0, 4), ('distributed', 0, 4), ('family', 0, 4), ('members', 0, 4), ('.', 2, 4), ('A', 1, 4), ('physical', 1, 4), ('artifact', 1, 4), ('dedicated', 1, 4), ('to', 1, 4), ('remote', 1, 4), ('family', 1, 4), ('members', 1, 4), ('makes', 1, 4), ('it', 1, 4), ('easier', 1, 4), ('to', 1, 4), ('chat', 1, 4), ('with', 1, 4), ('them', 1, 4), ('over', 1, 4), ('video', 1, 4), ('.', 1, 4), ('HomeProxy', 1, 4), ('combines', 1, 4), ('a', 1, 4), ('form', 1, 4), ('factor', 1, 4), ('designed', 1, 4), ('for', 1, 4), ('the', 1, 4), ('home', 1, 4), ('environment', 1, 4), ('with', 1, 4), ('a', 1, 4), ('``', 1, 4), ('no', 1, 4), ('-', 1, 4), ('touch', 1, 4), ("''", 1, 4), ('user', 1, 4), ('experience', 1, 4), ('and', 1, 4), ('an',

[('Social', 2, 4), ('networking', 2, 4), ('sites', 2, 4), ('(', 2, 4), ('SNSs', 2, 4), (')', 2, 4), ('offer', 2, 4), ('users', 2, 4), ('a', 2, 4), ('platform', 2, 4), ('to', 2, 4), ('build', 2, 4), ('and', 2, 4), ('maintain', 2, 4), ('social', 2, 4), ('connections', 2, 4), ('.', 2, 4), ('Understanding', 2, 4), ('when', 2, 4), ('people', 2, 4), ('feel', 2, 4), ('comfortable', 2, 4), ('sharing', 2, 4), ('information', 2, 4), ('about', 2, 4), ('themselves', 2, 4), ('on', 2, 4), ('SNSs', 2, 4), ('is', 2, 4), ('critical', 2, 4), ('to', 2, 4), ('a', 2, 4), ('good', 2, 4), ('user', 2, 4), ('experience', 2, 4), (',', 2, 4), ('because', 2, 4), ('self', 2, 4), ('-', 2, 4), ('disclosure', 2, 4), ('helps', 2, 4), ('maintain', 2, 4), ('friendships', 2, 4), ('and', 2, 4), ('increase', 2, 4), ('relationship', 2, 4), ('closeness', 2, 4), ('.', 2, 4), ('This', 0, 4), ('observational', 0, 4), ('research', 0, 4), ('develops', 0, 4), ('a', 0, 4), ('machine', 0, 4), ('learning', 0, 4), ('model', 0, 4), ('t

[('Crowd', 2, 4), ('workers', 2, 4), ('are', 2, 4), ('distributed', 2, 4), ('and', 2, 4), ('decentralized', 2, 4), ('.', 2, 4), ('While', 0, 4), ('decentralization', 0, 4), ('is', 0, 4), ('designed', 0, 4), ('to', 0, 4), ('utilize', 0, 4), ('independent', 0, 4), ('judgment', 0, 4), ('to', 0, 4), ('promote', 0, 4), ('high', 0, 4), ('-', 0, 4), ('quality', 0, 4), ('results', 0, 4), (',', 0, 4), ('it', 0, 4), ('paradoxically', 0, 4), ('undercuts', 0, 4), ('behaviors', 0, 4), ('and', 0, 4), ('institutions', 0, 4), ('that', 0, 4), ('are', 0, 4), ('critical', 0, 4), ('to', 0, 4), ('high', 0, 4), ('-', 0, 4), ('quality', 0, 4), ('work', 0, 4), ('.', 2, 4), ('Reputation', 0, 4), ('is', 0, 4), ('one', 0, 4), ('central', 0, 4), ('example', 0, 4), (':', 0, 4), ('crowdsourcing', 0, 4), ('systems', 0, 4), ('depend', 0, 4), ('on', 0, 4), ('reputation', 0, 4), ('scores', 0, 4), ('from', 0, 4), ('decentralized', 0, 4), ('workers', 0, 4), ('and', 0, 4), ('requesters', 0, 4), (',', 0, 4), ('but', 0, 4),

/anaconda3/lib/python3.6/site-packages/sklearn/metrics/classification.py:1135: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


In [9]:
print(df_groundtruth)

                             _id         content                     docID  \
0       5ab31d822bdc9e63125c9fa7           using  5ab31d802bdc9e63125c9ef5   
1       5ab31d822bdc9e63125c9fa8              an  5ab31d802bdc9e63125c9ef5   
2       5ab31d822bdc9e63125c9fa9        existing  5ab31d802bdc9e63125c9ef5   
3       5ab31d822bdc9e63125c9faa           crowd  5ab31d802bdc9e63125c9ef5   
4       5ab31d822bdc9e63125c9fab     marketplace  5ab31d802bdc9e63125c9ef5   
5       5ab31d822bdc9e63125c9fac               (  5ab31d802bdc9e63125c9ef5   
6       5ab31d822bdc9e63125c9fad          Amazon  5ab31d802bdc9e63125c9ef5   
7       5ab31d822bdc9e63125c9fae      Mechanical  5ab31d802bdc9e63125c9ef5   
8       5ab31d822bdc9e63125c9faf            Turk  5ab31d802bdc9e63125c9ef5   
9       5ab31d822bdc9e63125c9fb0               )  5ab31d802bdc9e63125c9ef5   
10      5ab31d822bdc9e63125c9fb1               ,  5ab31d802bdc9e63125c9ef5   
11      5ab31d822bdc9e63125c9fb2         explore  5ab31d802bdc9e